In [ ]:
# doesn't work, bug with mnist and loading dataset
# https://stackoverflow.com/questions/53142982/cannot-load-mnist-data-with-the-python-mnist-package-on-windows

import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# print ("TensorFlow version: " + tf.__version__)
import pickle
import numpy as np 

## tf 2.0 has diff way of using mnist data
## couldn't get this to work
## https://stackoverflow.com/a/61357344/13944490
# mnist = tf.keras.datasets.mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
## scales data between 0 and 1
# x_train = tf.keras.utils.normalize(x_train, axis=1) 
# x_test = tf.keras.utils.normalize(x_test, axis=1)
## tried to convert to float32, didn't work
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')

# node size doesn't have to be same
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100
hm_epochs = 10

# height by width
# https://stackoverflow.com/a/56230839/13944490 TF 2.0 doesn't use placeholders
x = tf.keras.backend.placeholder([None, 28 * 28], dtype='float')
y = tf.keras.backend.placeholder(dtype='float')

# x = tf.placeholder('float', [None, 28 * 28])
# y = tf.placeholder('float')

def neural_network_model(data):
    # 28 * 28 pixel image, going to n_nodes for hl1
    # randomly initialise weights to small, truncated normal variable with small stddev
    # set a bias so that all neurons have impact on output, so can tweak each neuron
    # bias doesn't have to be random, should be constant. number of bias units in hl1 is number of nodes hl1
    # bias needs to be in array
    # tf 2 needs compat.v1 https://stackoverflow.com/a/55143329/13944490
    # https://www.tensorflow.org/api_docs/python/tf/random/truncated_normal
    # https://www.tensorflow.org/api_docs/python/tf/constant
    # old code used hidden_1_layer = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
    #                                'biases': tf.Variable(tf.random_normal(n_nodes_hl1))}
    hidden_1_layer = {'weights': tf.Variable(tf.compat.v1.truncated_normal([28 * 28, n_nodes_hl1], stddev=0.1)),
                     'biases': tf.Variable(tf.constant(0.1, shape=[n_nodes_hl1]))}
    hidden_2_layer = {'weights': tf.Variable(tf.compat.v1.truncated_normal([n_nodes_hl1, n_nodes_hl2], stddev=0.1)),
                     'biases': tf.Variable(tf.constant(0.1, shape=[n_nodes_hl2]))}
    hidden_3_layer = {'weights': tf.Variable(tf.compat.v1.truncated_normal([n_nodes_hl2, n_nodes_hl3], stddev=0.1)),
                     'biases': tf.Variable(tf.constant(0.1, shape=[n_nodes_hl3]))}
    output_layer = {'weights': tf.Variable(tf.compat.v1.truncated_normal([n_nodes_hl3, n_classes], stddev=0.1)),
                     'biases': tf.Variable(tf.constant(0.1, shape=[n_classes]))}
    
    # (input_data * weights) + biases
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)  # activation function
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)  # activation function
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)  # activation function

    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    # https://stackoverflow.com/questions/55318273/tensorflow-api-v2-train-has-no-attribute-adamoptimizer
    optimizer = tf.optimizers.Adam()
    
    with tf.compat.v1.Session() as sess:
        # sess.run(tf.global_variables_initializer())
        # sess.run(tf.initalize_all_variables())
        
        # epoch is a feed fwd with backprop
        for epoch in range(hm_epochs):
            epoch_loss = 0

            for _ in range(int(mnist.train.num_examples / batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                
                epoch_loss += c
                
            print('Epoch ', epoch + 1, ' completed out of ', hm_epochs, ' loss: ', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy: ', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
train_neural_network(x)